In [1]:
# 1. 환경 설정
import argparse
import collections
import os
import random
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

In [2]:
kowiki_file = './data/kowiki.txt.zip'

In [3]:
# 환경 설정

args = {
    # random seed value
    'seed' : 1234,
    
    # cpu 또는 gpu 사용 여부 결정
    'device' : torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    
    # 말뭉치 파일
    'corpus' : kowiki_file
}

args = argparse.Namespace(**args)

In [4]:
print(args)

Namespace(seed=1234, device=device(type='cuda'), corpus='./data/kowiki.txt.zip')


In [5]:
# random seed 설정

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

In [6]:
# 파일 내용 확인

# wiki lines (라인 수) 확인
count = 0 
with zipfile.ZipFile(args.corpus) as z :
    with z.open('kowiki.txt') as f :
        for i, line in enumerate(f) :
            count += 1

print(count)

3724301


In [8]:
# wiki 내용 확인

with zipfile.ZipFile(args.corpus) as z :
    with z.open('kowiki.txt') as f :
        for i, line in enumerate(f) :
            if i >= 50 :
                break
            line = line.decode('utf-8').strip()
            print(line)

지미 카터
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.
1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.
카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.
카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다.
그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 1979년 백악관에서 양국 간의 평화조약으로 이끌어졌다. 또한 소련과 제2차 전략 무기 제한 협상에 조인했다.
카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.
그러나 주 이란 미국 대사관 인질 사건에서 인질 구출 실패를 이유로 1980년 대통령 선거에서 공화당의 로널드 레이건 후보에게 져 결국 재선에 실패했다. 또한 임기 말기에 터진 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을 내세웠다.
지미 카터는 대한민국과의 관계에서도 중요한 영향을 미쳤던 대통

In [9]:
# Sentencepiece
# 문장을 일정한 크기 subword 분할 기술
# "안녕하세요" >> "안녕", "하세요"
# 언어에 무관, 띄어쓰기 유무에 영향 없음
# 공백 문자 >> 심볼로 취급(이를 구분하기 위해 공백 문자를 특ㅈ겅 심볼로 변환)

In [10]:
# sentencepiece 설치
!pip install sentencepiece

                                              0.0/977.8 kB ? eta -:--:--
     ------------------------------------- 977.8/977.8 kB 60.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import sentencepiece as spm

In [16]:
def train_sentencepiece(corpus, prefix, vocab_size = 32000) :
    '''
    sentencepiece 이용, vocab 학습
    : param corpus : 학습할 말뭉치
    : param prefix : 저장ㅇ할 vocab 이름
    : param vocab_size : vocab 개수
    '''
    
    spm.SentencePieceTrainer.Train(
        f'--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}'+ # +7 : 특수문자 개수
        '--model_type=unigram' + # 단어 하나씩
        '--max_sentence_length=999999' + # 문장 최대 길이
        '--pad_id=0 --pad_piece=[PAD]' + # pad token 및 id 지정
        '--unk_id=1 --unk_piece=[UNK]' + # OOV(unknown token)및 id 지정
        '--bos_id=2 --bos_piece=[BOS]' + # begin of sequence token 및 id 지정
        '--eos_id=3 --eos_piece=[EOS]' + # end of sequence token 및 id 지정
        '--user_defined_symbols=[SEP],[CLS],[MASK]' + # 기타 추가 토큰 SEP:4, CLS:5, MASK:6
        '--input_sentence_size=20000' + # 말뭉치에서 샘플링해서 학습
        '--shuffle_input_sentence=True' # 샘플링한 말뭉치 shuffle
    )

In [17]:
os.listdir('./data/')

['kowiki.csv',
 'kowiki.model',
 'kowiki.txt',
 'kowiki.txt.zip',
 'kowiki.vocab',
 'ratings_test.json',
 'ratings_test.txt',
 'ratings_train.json',
 'ratings_train.txt']

In [18]:
# vocab 학습

train_sentencepiece('./data/kowiki.txt/kowiki.txt','kowiki_32000')
# voca size를 32000 인 파일 생성

In [19]:
# 생성 결과 확인
os.listdir('./')
# 'kowiki_32000.model'
# 'kowiki_32000.vocab'

['a_01_자연어전처리.ipynb',
 'a_02_언어모델_실용강의_딥러닝_구조와_컴파일_교육생_배포용.ipynb',
 'a_03_언어모델_자연어전처리.ipynb',
 'a_04_언어모델_감정분류.ipynb',
 'a_05_언어모델_언어생성모델.ipynb',
 'a_06_Attention mechanism.ipynb',
 'a_07언어모델_Tokenizer.ipynb',
 'b_01_PyTorch_Tensor.ipynb',
 'b_02_Pytorch_선형_회귀,_로지스틱_회귀_기본_구현.ipynb',
 'b_03_클래스로_선형_회귀,_로지스틱_회귀,_소프트맥스_회귀_구현하기.ipynb',
 'b_04_TF-IDF_MLP.ipynb',
 'data',
 'kowiki_32000.model',
 'kowiki_32000.vocab',
 'nsmc',
 'ratings_test.txt',
 'ratings_train.txt']

In [20]:
# 생성 파일 백업

corpus_dir = os.path.dirname(args.corpus)
print(os.listdir(corpus_dir))

['kowiki.csv', 'kowiki.model', 'kowiki.txt', 'kowiki.txt.zip', 'kowiki.vocab', 'ratings_test.json', 'ratings_test.txt', 'ratings_train.json', 'ratings_train.txt']


In [21]:
# load vocab

spm_vocab = spm.SentencePieceProcessor()
spm_vocab.load(os.path.join('./','kowiki_32000.model'))

True

In [25]:
# wiki spm tokenize

with zipfile.ZipFile(args.corpus) as z :
    with z.open('kowiki.txt') as f :
        for i, line in enumerate(f) :
            if i >= 5 :
                break
            
            line = line.decode('utf-8').strip()

            print(line)
            tokens = spm_vocab.encode_as_pieces(line)
            print(tokens)
            _ids = spm_vocab.encode_as_ids(line)
            print(_ids)
            
            print()

지미 카터
['▁지미', '▁카터']
[12655, 12701]

제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
['▁제임스', '▁얼', '▁"', '지', '미', '"', '▁카터', '▁주니어', '(', ',', '▁1924', '년', '▁10', '월', '▁1', '일', '▁~', '▁', ')', '는', '▁민주당', '▁출신', '▁미국', '▁39', '번', '째', '▁대통령', '▁', '(1977', '년', '▁~', '▁1981', '년', ')', '이다', '.']
[1878, 5628, 58, 28, 137, 41, 12701, 5031, 10, 4, 4094, 12, 94, 21, 37, 23, 111, 5, 9, 8, 2183, 1075, 161, 4681, 206, 579, 493, 5, 20821, 12, 111, 1881, 12, 9, 29, 3]

지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
['▁지미', '▁카터', '는', '▁조지아주', '▁섬', '터', '▁카운티', '▁', '플레인', '스', '▁마을에서', '▁태어났다', '.', '▁조지아', '▁공과대학교', '를', '▁졸업하였다', '.', '▁그', '▁후', '▁해군', '에', '▁들어가', '▁전함', '·', '원자력', '·', '잠수함', '의', '▁승무원', '으로', '▁', '일', '하였다', '.', '▁1953', '년', '▁미국', '▁해군', '▁대', '위', '로', '▁예편', '하였고', '▁이후', '▁땅콩', '·', '면', '화', '▁등을',

In [26]:
# 문자열을 token으로 변경

spm_vocab.encode_as_pieces('아름다운 대한민국 우리나라 금수강산')

['▁아름다운', '▁대한민국', '▁우리나라', '▁금', '수', '강', '산']

In [31]:
tokens = spm_vocab.encode_as_pieces('파이널 프로젝트를 진행중입니다. 잘 마무리 할 수 있도록 노력하겠습니다.')

In [32]:
# tokens  문자열로 복원

print(spm_vocab.decode_pieces(tokens))

파이널 프로젝트를 진행중입니다. 잘 마무리 할 수 있도록 노력하겠습니다.


In [33]:
# 문자열을 숫조라 분할

spm_vocab.encode_as_ids('아름다운 대한민국 우리나라 금수강산')

[4328, 258, 5299, 627, 65, 276, 115]

In [45]:
ids = spm_vocab.encode_as_ids('파이널 프로젝트를 진행중입니다. 잘 마무리 할 수 있도록 노력하겠습니다.')
ids

[11283,
 10931,
 1023,
 197,
 4865,
 3,
 436,
 3851,
 328,
 30,
 1332,
 1988,
 96,
 23966,
 3]

In [46]:
# 숫자를 문자열로 복원
spm_vocab.decode_ids(ids)

'파이널 프로젝트를 진행중입니다. 잘 마무리 할 수 있도록 노력하겠습니다.'

In [47]:
# token을 숫자로 변경
spm_vocab.piece_to_id(tokens)

[11283,
 10931,
 1023,
 197,
 4865,
 3,
 436,
 3851,
 328,
 30,
 1332,
 1988,
 96,
 23966,
 3]

In [48]:
# 숫자를 token으로 변경
spm_vocab.id_to_piece(ids)

['▁파이널',
 '▁프로젝트를',
 '▁진행',
 '중',
 '입니다',
 '.',
 '▁잘',
 '▁마무리',
 '▁할',
 '▁수',
 '▁있도록',
 '▁노력',
 '하',
 '겠습니다',
 '.']